In [2]:
from bs4 import BeautifulSoup
import requests
import sys

# Access page
cik = ' 0000072971'
type = '10-K'
dateb = '20180101'



In [3]:
# Obtain HTML for search page
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
edgar_resp = requests.get(base_url.format(cik, type, dateb))
edgar_str = edgar_resp.text



In [4]:
edgar_resp

<Response [200]>

In [5]:
edgar_str

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html lang="ENG">\n<head>\n<title>EDGAR Search Results</title>\n<link href="/include/interactive.css" rel="stylesheet" type="text/css" />\n<link rel="alternate" type="application/atom+xml" title="ATOM" href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0000072971&amp;type=10-K%25&amp;dateb=20180101&amp;owner=include&amp;count=40&amp;output=atom" />\n\n</head>\n<body style="margin: 0">\n<!-- SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo -->\n<noscript><iframe src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV"\nheight="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\nj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n\'//www.googletag

In [6]:
# Find the document link
doc_link = ''
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    print(len(cells))
    if len(cells) > 3:
        if '2015' in cells[3].text:
            doc_link = 'https://www.sec.gov' + cells[1].a['href']





0
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [7]:
doc_link

'https://www.sec.gov/Archives/edgar/data/72971/000007297115000449/0000072971-15-000449-index.htm'

In [8]:
# Exit if document link couldn't be found
if doc_link == '':
    print("Couldn't find the document link")
    sys.exit()

In [9]:
rows

[<tr>
 <th scope="col" width="7%">Filings</th>
 <th scope="col" width="10%">Format</th>
 <th scope="col">Description</th>
 <th scope="col" width="10%">Filing Date</th>
 <th scope="col" width="15%">File/Film Number</th>
 </tr>, <tr>
 <td nowrap="nowrap">10-K</td>
 <td nowrap="nowrap"><a href="/Archives/edgar/data/72971/000007297117000278/0000072971-17-000278-index.htm" id="documentsbutton"> Documents</a>  <a href="/cgi-bin/viewer?action=view&amp;cik=72971&amp;accession_number=0000072971-17-000278&amp;xbrl_type=v" id="interactiveDataBtn"> Interactive Data</a></td>
 <td class="small">Annual report [Section 13 and 15(d), not S-K Item 405]<br/>Acc-no: 0000072971-17-000278 (34 Act)  Size: 94 MB            </td>
 <td>2017-03-01</td>
 <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=001-02979&amp;owner=include&amp;count=40">001-02979</a><br/>17653045         </td>
 </tr>, <tr class="blueRow">
 <td nowrap="nowrap">10-K</td>
 <td nowrap="nowrap"><a href="/Archives

In [10]:
cells

[<td nowrap="nowrap">10-K</td>,
 <td nowrap="nowrap"><a href="/Archives/edgar/data/72971/0000072971-94-000003-index.html" id="documentsbutton"> Documents</a></td>,
 <td class="small">Annual report [Section 13 and 15(d), not S-K Item 405]<br>Acc-no: 0000072971-94-000003 Size: 384 KB            </br></td>,
 <td>1994-03-08</td>,
 <td nowrap="nowrap"><a href="/cgi-bin/browse-edgar?action=getcompany&amp;filenum=001-02979&amp;owner=include&amp;count=40">001-02979</a><br/>94515031         </td>]

In [11]:
doc_link


'https://www.sec.gov/Archives/edgar/data/72971/000007297115000449/0000072971-15-000449-index.htm'

In [12]:
# Obtain HTML for document page
doc_resp = requests.get(doc_link)
doc_str = doc_resp.text




In [13]:
# Find the XBRL link
xbrl_link = ''
soup = BeautifulSoup(doc_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile', summary='Data Files')
rows = table_tag.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 3:
        if 'INS' in cells[3].text:
            xbrl_link = 'https://www.sec.gov' + cells[2].a['href']



In [14]:
# Obtain XBRL text from document
xbrl_resp = requests.get(xbrl_link)
xbrl_str = xbrl_resp.text



In [18]:
# Find and print stockholder's equity
soup = BeautifulSoup(xbrl_str, 'html.parser')
tag_list = soup.find_all()

#This section of code creates a context table. 
#The context table is a dictionary of context names keys that reference dictionary values 
#containing date information for each context. For contexts with datetype of 'period' the table 
#contains the start and end date. For contexts with datetype of 'instant' the context
#contains the instant date of the context. All entries include a date and dateType value.
#For contexts with datetype of period, the date is equal to the enddate of the context.

contexts = {}

for tag in tag_list:
    if tag.name == 'xbrli:context':

#This section of code finds the start date of the context if it exists.
        start_date_tag = tag.find(name = 'xbrli:startdate')
        if start_date_tag == None:
            start_date = None
        else:
            start_date = start_date_tag.text

#This section of code finds the end date of the context if it exists.
        end_date_tag = tag.find(name = 'xbrli:enddate')
        if end_date_tag == None:
            end_date = None
        else:
            end_date = end_date_tag.text
            date = end_date_tag.text
            datetype = 'period'
#This section of code finds the instant date of the context if it exists.
        instant_date_tag = tag.find(name = 'xbrli:instant')
        if instant_date_tag != None:
            date = instant_date_tag.text
            datetype = 'instant'
#build a dictionary of date information within a dictionary of context titles
        dtinfo = {'date' : date, 'year' : date[0:4], 'datetype' : datetype, 'startdate' : start_date, 'enddate' : end_date}
        contexts[tag.attrs['id']] = dtinfo
    


In [23]:
# Find and print stockholder's equity
for tag in tag_list:
    if tag.name == 'us-gaap:stockholdersequity':
        year = contexts[tag.attrs['contextref']]['year']
        
        print(year + " Stockholder's equity: " + tag.text)

2013 Stockholder's equity: 170142000000
2013 Stockholder's equity: 170142000000
2014 Stockholder's equity: 184394000000
2014 Stockholder's equity: 184394000000
